install take ~25min；https://github.com/BioinfoMachineLearning/GCPNet-EMA

In [ ]:
from IPython.utils import io
import os
import sys
%shell rm -rf /opt/conda
%shell wget -q -P /tmp \
  https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh \
    && bash /tmp/Mambaforge-$(uname)-$(uname -m).sh -b -p /opt/conda \
    && rm /tmp/Mambaforge-$(uname)-$(uname -m).sh
PATH=%env PATH
%env PATH=/opt/conda/bin:{PATH}
%shell rm -rf /content/GCPNet-EMA
!git clone https://github.com/BioinfoMachineLearning/GCPNet-EMA.git
!cd /content/GCPNet-EMA
%shell mamba env create -f /content/GCPNet-EMA/cpu_environment.yaml #environment.yaml

In [ ]:
%%shell
cd /content/GCPNet-EMA/data/EMA
wget https://zenodo.org/record/10719475/files/ema_decoy_model.tar.gz
wget https://zenodo.org/record/10719475/files/ema_true_model.tar.gz
tar -xzf ema_decoy_model.tar.gz
tar -xzf ema_true_model.tar.gz
wget https://zhanggroup.org/TM-score/TMscore.cpp
g++ -static -O3 -ffast-math -lm -o TMscore TMscore.cpp

source activate GCPNet-EMA
python -V
cd /content/GCPNet-EMA
pip install -e .
# install ProDy separately to avoid a Biopython version conflict with Ankh
pip install prody==2.4.1
# uninstall protobuf (if installed) due to (potential) local GLIBCXX conflicts
pip uninstall -y protobuf
pip list

Download all checkpoints

In [ ]:
%%shell
wget -P checkpoints/ https://zenodo.org/record/10719475/files/structure_ema_finetuned_gcpnet_i2d5t9xh_best_epoch_106.ckpt
wget -P checkpoints/ https://zenodo.org/record/10719475/files/structure_denoising_pretrained_gcpnet.ckpt
wget -P checkpoints/ https://zenodo.org/record/10719475/files/structure_ema_finetuned_gcpnet_without_plddt_ije6iplr_best_epoch_055.ckpt
wget -P checkpoints/ https://zenodo.org/record/10719475/files/default_structure_ema_finetuned_gcpnet_without_plddt_or_esm_emb_p0p8c6pz_best_epoch_099.ckpt
wget -P checkpoints/ https://zenodo.org/record/10719475/files/structure_ema_finetuned_gcpnet_without_esm_emb_x8tjgsf4_best_epoch_027.ckpt

Edit configs/paths/default.yaml

In [ ]:
import sys

sys.path.append("/opt/conda/lib/python3.11/site-packages")
sys.path.append("/opt/conda/envs/GCPNet-EMA/lib/python3.10/site-packages")
sys.path.append("/content/GCPNet-EMA")
sys.path.append("/opt/conda/envs/GCPNet-EMA")

file_path = "/content/GCPNet-EMA/configs/paths/default.yaml"
line_number = 24
root_dir = "root_dir: ."  # ???
tmscore_path = "tmscore_exec_path: /content/GCPNet-EMA/data/EMA/TMscore"
lddt_path = "lddt_exec_path: /opt/conda/envs/GCPNet-EMA/bin/lddt"

# Read the file and modify the content
with open(file_path, "r") as file:
    lines = file.readlines()
    lines[3] = root_dir + "\n"
    lines[line_number - 1] = tmscore_path + "\n"
    lines[line_number] = lddt_path + "\n"

# Write the modified content back to the file
with open(file_path, "w") as file:
    file.writelines(lines)

In [ ]:
%%shell
source activate GCPNet-EMA
cd /content/GCPNet-EMA
python src/train.py experiment=/content/GCPNet-EMA/configs/model/gcpnet_ema.yaml